In [84]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

BASE_DIR = '../'


In [85]:
import os
import fiona
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
os.chdir(BASE_DIR + '/notebooks/')

In [86]:
def is_inter(id):
    if len(str(id)) > 1 and str(id)[0:2] == '00':
        return False
    return True

In [87]:
def summary_crash_rate(crashes, inter_count, non_inter_count):
    """                                                                                                                                                                 
    Info about intersections vs non intersections and their crash rate                                                                                                  
    """
    counts = {
        'inter': 0,
        'non_inter': 0,
        'no_match': 0,
        'inter_plus': 0,
        'non_inter_plus': 0,
    }

    for k, v in crashes.iteritems():

        if str(k) == '':
            # Sometimes crashes can't be snapped to a segment, probably due to bad data entry from the crash
            counts['no_match'] += 1
        elif is_inter(k):
            if int(v['count']) > 1:
                counts['inter_plus'] += 1
            counts['inter'] += 1
        else:
            if int(v['count']) > 1:
                counts['non_inter_plus'] += 1
            counts['non_inter'] += 1

    print "Number of intersections:\t\t\t\t" + str(inter_count)
    print "Number of non-intersections:\t\t\t\t" + str(non_inter_count)
    print "Percentage of segments that are intersections:\t\t" \
        + str(round(100 * float(inter_count)/float(inter_count + non_inter_count)))
    print ""

    print "Number of intersections with at least 1 crash:\t\t" + str(counts['inter']) 
    print 'Number of intersections with more than 1 crash:\t\t' + str(counts['inter_plus'])
    print "Number of non-intersections with at least 1 crash:\t" + str(counts['non_inter'])
    print "Number of non-intersections with more than 1 crash:\t" + str(counts['non_inter_plus'])
    print ""

    total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
    print "percent of all segments with crash:\t\t\t" + str(total_percent)

    # Percentage of intersections/non-intersections                                                                                                                     
    # that have at least one crash
    inter_percent = round(float(100 * counts['inter'])/float(inter_count))
    print "percent of intersections with crash:\t\t\t" + str(inter_percent)

    non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
    print "percentage of non-intersections with crash:\t\t" + str(non_inter_percent)

    print "percentage of intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['inter_plus'])/float(inter_count)))

    print "percentage of non-intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['non_inter_plus'])/float(non_inter_count)))


In [88]:
def summary_concern_counts(crashes, concerns):
    """                                                                                                                                                                 
    What percentage of intersections with concerns had crashes                                                                                                          
    at varying counts of concerns?                                                                                                                                      
    """

    matching = {}
    # Go through each concern location
    # Increment counts for crash/no crash intersection/no intersection at this location
    for id, d in concerns.iteritems():
        if d['count'] not in matching.keys():
            matching[d['count']] = {
                'inter': {'crash': 0, 'no_crash': 0},
                'non_inter': {'crash': 0, 'no_crash': 0}
            }
        if is_inter(id):
            key = 'inter'
        else:
            key = 'non_inter'

        if id in crashes.keys():
            matching[d['count']][key]['crash'] += 1
        else:
            matching[d['count']][key]['no_crash'] += 1

    sorted_matching = sorted(matching.items())

    results = []
    for key, value in sorted_matching:

        # Do the 1+,2+ stats as well as 1, 2                                                                                                                            
        # Still need to break it out by int/non-int                                                                                                                     
        counts = {
            'total': value['inter']['crash'] + value['inter']['no_crash'] \
                + value['non_inter']['crash'] + value['non_inter']['no_crash'],
            'crashes': value['inter']['crash'] + value['non_inter']['crash'],
            'inters_total': value['inter']['crash'] + value['inter']['no_crash'], # Count of intersections with a concern
            'inters_crashes': value['inter']['crash'],
            'non_inters_total': value['non_inter']['crash'] + value['non_inter']['no_crash'],
            'non_inters_crashes':  value['non_inter']['crash'],
        }
        # Add all the data for segments with more complaints than the current complaint # we're on
        for key2, value2 in sorted_matching[key:len(sorted_matching)]:

            if key2 > key:
                counts['total'] += value2['inter']['crash'] + value2['inter']['no_crash'] \
                    + value2['non_inter']['crash'] + value2['non_inter']['no_crash']
                counts['crashes'] += value2['inter']['crash'] + value2['non_inter']['crash']
                counts['inters_total'] += value2['inter']['crash'] + value2['inter']['no_crash']
                counts['inters_crashes'] += value2['inter']['crash']
                counts['non_inters_total'] += value2['non_inter']['crash'] + value2['non_inter']['no_crash']
                counts['non_inters_crashes'] += value2['non_inter']['crash']
        total_percent_v0 = round(100 * float(counts['crashes'])/float(counts['total']))
        inter_percent_v0 = round(100 * float(counts['inters_crashes'])/float(counts['inters_total'])) if counts['inters_total'] else 0
        non_inter_percent_v0 = round((100 * float(counts['non_inters_crashes'])/float(counts['non_inters_total'])) if counts['non_inters_total'] else 0)
        results.append([
            key,
            total_percent_v0,
            counts['total'], # total count      
            inter_percent_v0,
            counts['inters_total'], # total # of intersections with this many or more complaints
            non_inter_percent_v0,
            counts['non_inters_total'], # total # of non-intersections with this many or more complaints
            round(100 * float(counts['inters_total'])/float(counts['total']))
        ])

    results = results[0:5]

    display(pd.DataFrame(results, columns=[
        '# of concerns at this segment',
        '% of segments w/ v0 complaint w/ crash',
        'total # of segments with this many or more concerns',
        '% of intersections with this many or more concerns with a crash',
        '# of intersections with this many concerns',
        '% of non-intersections with this many concerns with a crash',
        '# of non-intersections with this many concerns',
        '% of these locations that were at an intersection'
    ]))

In [97]:
def concerns_by_type(concerns, concern_data, crashes, category_field='REQUEST_TYPE'):
    # For intersections with a specific complaint type:                                                                                                                 
    #    what percentage had a crash?                                                                                                                                   
    requests = {}
    for data in concern_data:
        if data[category_field] not in requests.keys():
            requests[data[category_field]] = 1

    requests = {}
    all_unique = []

    for k, v in concerns.iteritems():
        unique_requests = {}
        for request in v[category_field]:
            # Clean up badly formatted request types                                                                                                                    
            vals = request.split('nbsp;')
            if len(vals) > 1:
                request = vals[1]

            if request not in unique_requests.keys():
                unique_requests[request] = 0
            unique_requests[request] += 1

            if request not in requests.keys():
                requests[request] = {
                    'crashes': 0,
                    'total': 0,
                    'inter_crashes': 0,
                    'inter_total': 0,
                    'non_inter_crashes': 0,
                    'non_inter_total': 0,
                    'ped_inter_crashes': 0,
                    'ped_non_inter_crashes': 0
                }
            if str(k) in crashes.keys():
                requests[request]['crashes'] += 1
            requests[request]['total'] += 1
            if is_inter(k):
                if str(k) in crashes.keys():
                    requests[request]['inter_crashes'] += 1
                requests[request]['inter_total'] += 1
            else:
                if str(k) in crashes.keys():
                    requests[request]['non_inter_crashes'] += 1
                requests[request]['non_inter_total'] += 1


        for key, value in unique_requests.iteritems():
            if value > 1:
                all_unique.append([key, value])

    by_type = {}
    for k, v in all_unique:
        if k not in by_type.keys():
            by_type[k] = 0
        by_type[k] += 1

    results = []
    for k, v in requests.iteritems():
        if requests[k]['total'] >= 100:
            total_percent = round(100 * float(requests[k]['crashes'])/float(requests[k]['total']))
            inter_percent = round(100 * float(requests[k]['inter_crashes'])/float(requests[k]['inter_total']))
            non_inter_percent = round(100 * float(requests[k]['non_inter_crashes'])/float(requests[k]['non_inter_total']) if requests[k]['non_inter_total'] else 0)
            results.append([
                k,
                by_type[k],
                total_percent,
                requests[k]['crashes'],
                requests[k]['total'],
                inter_percent,
                requests[k]['inter_crashes'],
                requests[k]['inter_total'],
                non_inter_percent,
                requests[k]['non_inter_crashes'],
                requests[k]['non_inter_total'],
                round(100 * float(requests[k]['inter_total'])/float(requests[k]['total']))
            ])


    df = pd.DataFrame(results, columns=[
        'Request type',
        '# of requests of this type that appear more than once at a segment',
        '% of segments with this type of request with a crash',
        '# of segments with this type of request with a crash',
        '# of segments with this type of a request total',
        '% of intersections with this type of request with a crash',
        '# of intersections with this type of request with a crash',
        '# of intersections with this type of request total',
        '% of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request total',
        '% of complaint locations that were intersections'
        #'% of intersections with this type of request with a ped crash',
        #'# of ped crashes in intersection with this request',
        #'# of ped crashes in non-intersection with this request'
    ])
    df.sort_values(by="# of segments with this type of a request total", ascending=0)
    display(df)


In [98]:
def get_analysis_for_city(datadir, category_field='REQUESTTYPE'):
    
    DATA_FP = BASE_DIR + '/' + datadir + '/processed/'
    MAP_FP = DATA_FP + 'maps/'
    crash_data, crashes = util.group_json_by_location(DATA_FP + 'crash_joined.json')

    concern_data, concerns = util.group_json_by_location(DATA_FP + 'concern_joined.json',
                                        otherfields=[category_field])
    inters = util.read_segments(MAP_FP, get_non_inter=False)
    non_inters = util.read_segments(MAP_FP, get_inter=False)
    inter_count = len(inters[0])
    non_inter_count = len(non_inters[0])
    
    summary_crash_rate(crashes, inter_count, non_inter_count)
    summary_concern_counts(crashes, concerns)
    concerns_by_type(concerns, concern_data, crashes, category_field)
    

In [99]:
get_analysis_for_city('osm-data')

Read in 7845 intersection, 0 non-intersection segments
Read in 0 intersection, 15434 non-intersection segments
Number of intersections:				7845
Number of non-intersections:				15434
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		1769
Number of intersections with more than 1 crash:		832
Number of non-intersections with at least 1 crash:	1610
Number of non-intersections with more than 1 crash:	428

percent of all segments with crash:			15.0
percent of intersections with crash:			23.0
percentage of non-intersections with crash:		10.0
percentage of intersections with more than 1 crash:	11.0
percentage of non-intersections with more than 1 crash:	3.0


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,31.0,4345,44.0,1981,21.0,2364,46.0
1,2,44.0,1596,58.0,814,28.0,782,51.0
2,3,52.0,804,68.0,430,34.0,374,53.0
3,4,60.0,474,73.0,271,43.0,203,57.0
4,5,68.0,308,81.0,181,50.0,127,59.0


,Request type,# of requests of this type that appear more than once at a segment,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total,% of complaint locations that were intersections
0,people speed,126,29.0,355,1226,41.0,143,352,24.0,212,874,29.0
1,people double park their vehicles,90,35.0,218,631,49.0,79,161,30.0,139,470,26.0
2,there are no bike facilities or they need main...,9,34.0,49,143,66.0,19,29,26.0,30,114,20.0
3,bike facilities don't exist or need improvement,97,40.0,460,1136,64.0,194,303,32.0,266,833,27.0
4,it’s hard to see / low visibility,31,40.0,172,429,47.0,140,296,24.0,32,133,69.0
5,people cross away from the crosswalks,33,44.0,128,293,57.0,80,141,32.0,48,152,48.0
6,people don't yield while going straight,37,48.0,173,361,55.0,138,253,32.0,35,108,70.0
7,people don't yield while turning,88,60.0,386,644,68.0,349,515,29.0,37,129,80.0
8,the roadway surface needs improvement,27,40.0,111,277,61.0,62,102,28.0,49,175,37.0
9,sidewalks/ramps don't exist or need improvement,32,40.0,130,329,55.0,72,130,29.0,58,199,40.0


In [95]:
get_analysis_for_city('osm-dc')

Read in 7480 intersection, 0 non-intersection segments
Read in 0 intersection, 14901 non-intersection segments
Number of intersections:				7480
Number of non-intersections:				14901
Percentage of segments that are intersections:		33.0

Number of intersections with at least 1 crash:		4675
Number of intersections with more than 1 crash:		3739
Number of non-intersections with at least 1 crash:	10075
Number of non-intersections with more than 1 crash:	8497

percent of all segments with crash:			66.0
percent of intersections with crash:			63.0
percentage of non-intersections with crash:		68.0
percentage of intersections with more than 1 crash:	50.0
percentage of non-intersections with more than 1 crash:	57.0


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,87.0,3342,86.0,1652,88.0,1690,49.0
1,2,93.0,1023,93.0,531,92.0,492,52.0
2,3,95.0,422,96.0,245,93.0,177,58.0
3,4,99.0,206,100.0,129,97.0,77,63.0
4,5,100.0,115,100.0,74,100.0,41,64.0


,Request type,# of requests of this type that appear more than once at a segment,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total,% of complaint locations that were intersections
0,Other Driving Issue,62,92.0,574,625,90.0,280,310,93.0,294,315,50.0
1,Failure to stop for pedestrians,79,90.0,656,731,91.0,510,562,86.0,146,169,77.0
2,Other Walking Issue,25,88.0,310,353,84.0,147,174,91.0,163,179,49.0
3,Double parking,24,95.0,233,246,93.0,25,27,95.0,208,219,11.0
4,Not enough time to cross,17,95.0,133,140,96.0,108,112,89.0,25,28,80.0
5,Poor visibility,22,83.0,238,287,84.0,161,191,80.0,77,96,67.0
6,Red light running,34,95.0,331,350,96.0,260,270,89.0,71,80,77.0
7,Blocking the crosswalk,6,94.0,123,131,99.0,93,94,81.0,30,37,72.0
8,Cyclist behavior,4,97.0,113,117,98.0,60,61,95.0,53,56,52.0
9,Jaywalking,13,97.0,170,175,99.0,96,97,95.0,74,78,55.0


In [100]:
get_analysis_for_city('osm-cambridge', category_field='issue_type')

Read in 1287 intersection, 0 non-intersection segments
Read in 0 intersection, 2456 non-intersection segments
Number of intersections:				1287
Number of non-intersections:				2456
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		882
Number of intersections with more than 1 crash:		677
Number of non-intersections with at least 1 crash:	1012
Number of non-intersections with more than 1 crash:	572

percent of all segments with crash:			51.0
percent of intersections with crash:			69.0
percentage of non-intersections with crash:		41.0
percentage of intersections with more than 1 crash:	53.0
percentage of non-intersections with more than 1 crash:	23.0


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,57.0,3155,73.0,1157,47.0,1998,37.0
1,2,59.0,2709,75.0,988,50.0,1721,36.0
2,3,62.0,2322,78.0,834,53.0,1488,36.0
3,4,65.0,1969,81.0,693,56.0,1276,35.0
4,5,67.0,1663,83.0,563,59.0,1100,34.0


,Request type,# of requests of this type that appear more than once at a segment,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total,% of complaint locations that were intersections
0,Missed Trash Pickup,138,67.0,496,745,86.0,160,186,60.0,336,559,25.0
1,Sidewalk Defect,334,70.0,1235,1762,85.0,458,542,64.0,777,1220,31.0
2,Traffic Sign Complaint,135,73.0,627,861,85.0,385,452,59.0,242,409,52.0
3,Other,488,73.0,1819,2504,86.0,781,911,65.0,1038,1593,36.0
4,Missed Compost Pickup (Green Bin),71,67.0,301,448,88.0,118,134,58.0,183,314,30.0
5,Streetlight Defect,90,65.0,343,525,84.0,159,190,55.0,184,335,36.0
6,Tree Pruning Request,148,67.0,568,851,84.0,205,245,60.0,363,606,29.0
7,DPW Bike Tagged (Internal),29,75.0,158,211,88.0,71,81,67.0,87,130,38.0
8,Missed Holiday Tree Pickup,24,56.0,98,175,72.0,26,36,52.0,72,139,21.0
9,Rodent Sighting,60,67.0,275,411,90.0,111,123,57.0,164,288,30.0


In [92]:
# Why are there things like turning complaints at non-intersections?
# Pedestrian vs. bike vs. car
# Compare counts against ATR/TMC volume counts
# Parsing extra field's text?